In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
inp = torch.rand(10,1,160,160,36)

In [3]:
inp.view(inp.shape[0],-1).shape

torch.Size([10, 921600])

In [4]:
conv1 = nn.Sequential(
                nn.Conv3d(1, 4, kernel_size=(7, 7, 7), padding=3),
                nn.ReLU(),
                nn.MaxPool3d((4, 4, 4)),
                )
conv2 = nn.Sequential(
                nn.Conv3d(4, 4, kernel_size=(5, 5, 5), padding=2),
                nn.ReLU(),
                nn.MaxPool3d((2, 2, 2)),
                )
conv3 = nn.Sequential(
                nn.Conv3d(4, 1, kernel_size=(3, 3, 3), padding=1),
                nn.ReLU(),
                nn.MaxPool3d((2, 2, 2)),
                )

In [5]:
print(inp.shape)
out = conv1(inp)
print(out.shape)
out = conv2(out)
print(out.shape)
out = conv3(out)
print(out.shape)

torch.Size([10, 1, 160, 160, 36])
torch.Size([10, 4, 40, 40, 9])
torch.Size([10, 4, 20, 20, 4])
torch.Size([10, 1, 10, 10, 2])


In [6]:
deconv1 = nn.Sequential(nn.ConvTranspose3d(1, 4, kernel_size= (3,3,3), padding=1),
                       nn.Upsample(size=(20,20,4)))

deconv2 = nn.Sequential(nn.ConvTranspose3d(4, 4, kernel_size= (5,5,5), padding=2),
                       nn.Upsample(size=(40,40,9)))

deconv3 = nn.Sequential(nn.ConvTranspose3d(4, 1, kernel_size= (7,7,7), padding=3),
                       nn.Upsample(size=(160,160,36)))

In [7]:
out = deconv1(out)
print(out.shape)
out = deconv2(out)
print(out.shape)
out = deconv3(out)
print(out.shape)

torch.Size([10, 4, 20, 20, 4])
torch.Size([10, 4, 40, 40, 9])
torch.Size([10, 1, 160, 160, 36])


In [8]:
num_classes = 5

class CNN_Enc_Dec(nn.Module):
    def __init__(self):
        super(CNN_Enc_Dec, self).__init__()
        
        self.conv1 = nn.Sequential(
                nn.Conv3d(1, 4, kernel_size=(7, 7, 7), padding=3),
                nn.ReLU(),
                nn.MaxPool3d((4, 4, 4)),
                )
        self.conv2 = nn.Sequential(
                nn.Conv3d(4, 4, kernel_size=(5, 5, 5), padding=2),
                nn.ReLU(),
                nn.MaxPool3d((2, 2, 2)),
                )
        self.conv3 = nn.Sequential(
                nn.Conv3d(4, 1, kernel_size=(3, 3, 3), padding=1),
                nn.ReLU(),
                nn.MaxPool3d((2, 2, 2)),
                )
        self.deconv1 = nn.Sequential(nn.ConvTranspose3d(1, 4, kernel_size= (3,3,3), padding=1),
                       nn.Upsample(size=(20,20,4)))

        self.deconv2 = nn.Sequential(nn.ConvTranspose3d(4, 4, kernel_size= (5,5,5), padding=2),
                       nn.Upsample(size=(40,40,9)))

        self.deconv3 = nn.Sequential(nn.ConvTranspose3d(4, 1, kernel_size= (7,7,7), padding=3),
                       nn.Upsample(size=(160,160,36)))
        

    def forward(self, x):
        # Set 1
        out = self.conv1(x)
        out = self.conv2(out)
        embed = self.conv3(out)
        out = self.deconv1(embed)
        out = self.deconv2(out)
        out = self.deconv3(out)
        
        return embed, out

In [9]:
model = CNN_Enc_Dec()
embed, out = model(inp)
print(embed.shape)
print(out.shape)

torch.Size([10, 1, 10, 10, 2])
torch.Size([10, 1, 160, 160, 36])


In [23]:
import os
import nibabel as nib
import pandas as pd

dirname = 'ds000113b_R2.0.1/'
brain_volumes = []
class_labels = []
classes = ['ambient', 'symphonic', 'country', 'rocknroll','metal' ]
for subj in os.listdir(dirname):
    for run in range(1,9):
        path = dirname+"/"+subj+"/func" + "/" + subj + '_task-auditoryperception_run-0'+ str(run)                     
        img = nib.load(path + '_bold.nii.gz')
        data = img.get_fdata()

        df = pd.read_csv(path + '_events.tsv', sep='\t')
        genres = df['genre'].values
        run_volumes = df['run_volume'].values
        
        for start, genre in zip(run_volumes, genres):
            volume = data[:,:,:,start:start+6].mean(axis=-1)
            brain_volumes.append(volume)
            class_labels.append(classes.index(genre))
    break

In [24]:
len(brain_volumes), len(class_labels)

(200, 200)

In [25]:
brain_volumes[0].shape

(160, 160, 36)

In [26]:
# import matplotlib.pyplot as plt
# a = brain_volumes[0]
# plt.imshow(a[:,:,18].T, cmap='Greys_r')
# print(data.shape)

In [46]:
from torch.utils.data import TensorDataset, DataLoader, Dataset
import numpy as np

class BrainDataset(Dataset):
    def __init__(self, brain_volumes, class_labels):
        self.brain_volumes = brain_volumes
        self.class_labels = class_labels

    def __len__(self):
        return len(self.class_labels)

    def __getitem__(self, idx):
        temp_img = brain_volumes[idx]
        ind = np.where(brain_volumes[idx] < 200)
        temp_img[ind] = 0
        temp_img = temp_img / 2200
        return temp_img, self.class_labels[idx]


In [47]:
# train_len = int(0.8 * len(brain_volumes))
# train_dataset = BrainDataset(brain_volumes[:train_len], class_labels[:train_len])
# val_dataset = BrainDataset(brain_volumes[train_len:], class_labels[train_len:])

dataset = BrainDataset(brain_volumes, class_labels)

In [48]:
train_dataset , val_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])

In [49]:
train_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=24)

In [50]:
for i, data in enumerate(train_loader):
    print(data[0].shape)
    print(data[1].shape)
    break

torch.Size([24, 160, 160, 36])
torch.Size([24])


In [55]:
def train_one_epoch(model, optimizer, dataloader,epoch):
    model.train()
    losses = []
    accs = []
    binaccs = []
    
    for i, (brain_volume, label) in enumerate(dataloader):
        optimizer.zero_grad()
        brain_volume = brain_volume.unsqueeze(1)
        
        brain_volume = brain_volume.to(device)
        label = label.to(device)
        
        
        embed, pred = model(brain_volume)
        loss = loss_fn(pred, brain_volume)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
#         pred_labels = torch.argmax(pred, axis=1)
        
#         acc = (pred_labels == label).sum() / len(label)
#         accs.append(acc.item())
        
#         pred_labels[pred_labels <=1] = 0
#         pred_labels[pred_labels > 1] = 1
#         label[label <=1 ]= 0 
#         label[label > 1] = 1
        
#         binacc = (pred_labels == label).sum() / len(label)
#         binaccs.append( binacc.item() )
        
        print("Epoch ", epoch,": ", i, "/", len(dataloader), "loss :", loss.item(), end='\r')
    print("train acc", np.mean(accs))
    print("train binary acc", np.mean(binaccs))
    return np.mean(losses), np.mean(accs)

def validate(model, optimizer, dataloader,epoch):
    model.eval()
    losses = []
    accs = []
    binaccs = []
    for i, (brain_volume, label) in enumerate(dataloader):
        brain_volume = brain_volume.unsqueeze(1)
        
        brain_volume = brain_volume.to(device)
        label = label.to(device)
         
        embed, pred = model(brain_volume)
        loss = loss_fn(pred, brain_volume)

#         pred_labels = torch.argmax(pred, axis=1)
        
#         acc = (pred_labels == label).sum() / len(label)
#         accs.append(acc.item())
        
#         losses.append(loss.item())
        
#         pred_labels[pred_labels <=1] = 0
#         pred_labels[pred_labels > 1] = 1
#         label[label <=1 ]= 0 
#         label[label > 1] = 1
#         binacc = (pred_labels == label).sum() / len(label)
#         binaccs.append( binacc.item() )
        
    print("val acc", np.mean(accs))
    print("val binary acc", np.mean(binaccs))
    
    plt.matshow(pred[0][0][:,:,8])
    plt.show()
    return np.mean(losses), np.mean(accs)

In [56]:
model = CNN_Enc_Dec()
device = torch.device('cuda')
model.to(device)
model.double()

# loss_fn = nn.CrossEntropyLoss()
def loss_fn(out, target):
    return torch.mean((out.view(out.shape[0], -1) - target.view(target.shape[0], -1))**2)

lr = 1e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-6)
num_epochs = 10
train_losses = []
val_losses = []
val_accs = []
train_accs = []
for epoch in range(num_epochs):
    loss, acc = train_one_epoch(model, optimizer, train_loader, epoch)
    val_loss, val_acc = validate(model, optimizer, val_loader, epoch)
    train_losses.append(loss)
    train_accs.append(acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print("---------------")

train acc nan / 7 loss : 0.016944581818808796
train binary acc nan


/home2/vijay.vignesh/miniconda3/envs/tdl/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home2/vijay.vignesh/miniconda3/envs/tdl/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


val acc nan
val binary acc nan


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_losses, label="train")
plt.plot(val_losses, label="val")
plt.legend()

In [ ]:
plt.plot(val_accs, label="val acc")
plt.plot(train_accs, label="train acc")
plt.legend()